# Cortex - A MRI Convolutional Neural Network 



## Overview


## Training

### Dependencies.

We are going to lean heavily on `tensorflow` for the training, `matplotlib` for the visualisation, `pandas` for wrangling.

In [32]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
#---------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#---------------------------------------
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from typing import List, Dict, Tuple
#---------------------------------------
import warnings
warnings.filterwarnings("ignore")

### Constants, Paths and Training Params

Defines a constants 


In [ ]:
DATA_DIR: str = os.path.join(os.getcwd(), "data")

MODEL_DIR: str = os.path.join(os.getcwd(), "models")
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

MODEL_TYPE:str = "cnn"

EPOCHS: int = 50
PATIENCE: int = int(EPOCHS * 0.1)

LEARNING_RATE: float = 0.001
BETA1: float = 0.95
BETA2: float = 0.999

RANDOM_STATE: int = 42
IMG_SIZE: Tuple[int,int] = (149,149)
BATCH_SIZE: int = 32


In [34]:
def init_model_dir(model_type, model_name, base_dir="models"):
    #date_str: str = datetime.now().strftime("%Y%m%d-%H%M")
    dir_path: str = os.path.join(base_dir, model_type, f"{model_name}")
    os.makedirs(dir_path, exist_ok=True)
    return dir_path

model_dir: str = init_model_dir(MODEL_TYPE.lower(), f"cortex-{MODEL_TYPE.lower()}-model")

In [35]:
for device in tf.config.list_physical_devices():
    if device.device_type == 'GPU':
        tf.config.experimental.set_memory_growth(device, True)
        print(f"Using GPU: {device.name}")
    else:
        print(f"Using CPU: {device.name}")

print("TensorFlow version:", tf.__version__)

Using CPU: /physical_device:CPU:0
Using GPU: /physical_device:GPU:0
TensorFlow version: 2.16.2


In [36]:
train_dir: str = os.path.join(DATA_DIR, "train")
test_dir: str = os.path.join(DATA_DIR, "test")

if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Data directory '{DATA_DIR}' does not exist.")
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"Training data directory '{train_dir}' does not exist.")
if not os.path.exists(test_dir):
    raise FileNotFoundError(f"Test data directory '{test_dir}' does not exist.")

train_images: List[str] = glob(os.path.join(train_dir, "*/*.jpg"))
test_images: List[str] = glob(os.path.join(test_dir, "*/*.jpg"))

if not train_images:
    raise FileNotFoundError(f"No training images found in '{train_dir}'.")
if not test_images:
    raise FileNotFoundError(f"No test images found in '{test_dir}'.")

print(f"Found {len(train_images)} training images and {len(test_images)} test images.")

Found 5712 training images and 1311 test images.


In [37]:
def dataclassifier(path: str) -> pd.DataFrame:
    records:List[Dict[str, str]] = []
    img_paths: List[str] = glob(os.path.join(path, "*/*.jpg"))
    for img_path in img_paths:
        record: dict = {
             "img_path": img_path,
            "level_1_class": "normal" if img_path.split(os.sep)[-2] == "notumor" else "abnormal",
            "level_2_class": img_path.split(os.sep)[-2]}
        records.append(record)
    df: pd.DataFrame = pd.DataFrame(records)
    return df

df_trn: pd.DataFrame = dataclassifier(train_dir)
df_tst: pd.DataFrame = dataclassifier(test_dir)

print(f"Training DataFrame shape: {df_trn.shape}")
print(f"Test DataFrame shape: {df_tst.shape}")

Training DataFrame shape: (5712, 3)
Test DataFrame shape: (1311, 3)


In [38]:
class Generator:
    def __init__(self, df: pd.DataFrame ,x_col:str,y_col:str, img_size: Tuple[int, int], batch_size: int, shuffle: bool = True, randomize: bool = False):
        self.df = df
        self.img_size = img_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.randomize = randomize
        self.x_col = x_col
        self.y_col = y_col
        
        if randomize:
            self.gen = self.flow_random()
        else:
            self.gen = self.flow()


    def flow_random(self) -> tf.keras.utils.Sequence:
        datagen: ImageDataGenerator = ImageDataGenerator(rescale=1./255)
        gen = datagen.flow_from_dataframe(
            dataframe = self.df,
            x_col = self.x_col,
            y_col = self.y_col,
            target_size = self.img_size,
            batch_size = self.batch_size,
            class_mode = 'categorical',
            width_shift_range=0.2,
            height_shift_range=0.2,
            rotation_range=25,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            brightness_range=[0.8, 1.2],
            seed=RANDOM_STATE,
        )
        return gen
    def flow(self) -> tf.keras.utils.Sequence:
        datagen: ImageDataGenerator = ImageDataGenerator(rescale=1./255)
        gen = datagen.flow_from_dataframe(
            dataframe = self.df,
            x_col = self.x_col,
            y_col = self.y_col,
            target_size = self.img_size,
            batch_size = self.batch_size,
            class_mode = 'categorical',
        )
        return gen


In [39]:
train_lvl_1_gen = Generator(    df=df_trn,
    x_col="img_path",
    y_col="level_1_class",
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    randomize=True
)
test_lvl_1_gen = Generator(
    df=df_tst,
    x_col="img_path",
    y_col="level_1_class",
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    randomize=False
)
train_lvl_2_gen = Generator(
    df=df_trn,
    x_col="img_path",
    y_col="level_2_class",
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    randomize=True
)
test_lvl_2_gen = Generator(
    df=df_tst,
    x_col="img_path",
    y_col="level_2_class",
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    randomize=False
)

Found 5712 validated image filenames belonging to 2 classes.
Found 1311 validated image filenames belonging to 2 classes.
Found 5712 validated image filenames belonging to 4 classes.
Found 1311 validated image filenames belonging to 4 classes.


In [40]:
n_types_l1: int = len(train_lvl_1_gen.gen.class_indices)

inputs = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), name='input')

x = Conv2D(32, (4, 4), activation="relu", kernel_regularizer=l2(0.001), name='conv_1')(inputs)
x = MaxPooling2D(pool_size=(3, 3), name='pool_1')(x)

x = Conv2D(64, (4, 4), activation="relu", kernel_regularizer=l2(0.001), name='conv_2')(x)
x = MaxPooling2D(pool_size=(3, 3), name='pool_2')(x)

x = Conv2D(128, (4, 4), activation="relu", kernel_regularizer=l2(0.001), name='conv_3')(x)
x = MaxPooling2D(pool_size=(3, 3), name='pool_3')(x)

x = Conv2D(128, (4, 4), activation="relu", kernel_regularizer=l2(0.001), name='conv_4')(x)
# No pooling here to match your original model
x = Flatten(name='flatten')(x)

x = Dense(512, activation="relu", kernel_regularizer=l2(0.001), name='fc_1')(x)
x = Dropout(0.5, seed=RANDOM_STATE, name='dropout')(x)

outputs = Dense(n_types_l1, activation="softmax", name='output')(x)

model_lvl_1 = Model(inputs=inputs, outputs=outputs, name='cortex_lvl1')


In [41]:
n_types_l2: int = len(train_lvl_2_gen.gen.class_indices)

inputs = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), name='input')

x = Conv2D(32, (4, 4), activation="relu", kernel_regularizer=l2(0.001), name='conv_1')(inputs)
x = MaxPooling2D(pool_size=(3, 3), name='pool_1')(x)

x = Conv2D(64, (4, 4), activation="relu", kernel_regularizer=l2(0.001), name='conv_2')(x)
x = MaxPooling2D(pool_size=(3, 3), name='pool_2')(x)

x = Conv2D(128, (4, 4), activation="relu", kernel_regularizer=l2(0.001), name='conv_3')(x)
x = MaxPooling2D(pool_size=(3, 3), name='pool_3')(x)

x = Conv2D(128, (4, 4), activation="relu", kernel_regularizer=l2(0.001), name='conv_4')(x)
# No pooling here to match your original model
x = Flatten(name='flatten')(x)

x = Dense(512, activation="relu", kernel_regularizer=l2(0.001), name='fc_1')(x)
x = Dropout(0.5, seed=RANDOM_STATE, name='dropout')(x)

outputs = Dense(n_types_l2, activation="softmax", name='output')(x)

model_lvl_2 = Model(inputs=inputs, outputs=outputs, name='cortex_lvl2')



In [42]:


optimizer_lvl_1 = Adam(learning_rate = LEARNING_RATE, beta_1=BETA1, beta_2=BETA2)
optimizer_lvl_2 = Adam(learning_rate=LEARNING_RATE, beta_1=BETA1, beta_2=BETA2)

model_lvl_1.compile(optimizer=optimizer_lvl_1, loss='categorical_crossentropy', metrics=['accuracy'])
model_lvl_2.compile(optimizer=optimizer_lvl_2, loss='categorical_crossentropy', metrics=['accuracy'])

train_model_lvl_1: tf.keras.callbacks.Callback = EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE,
    restore_best_weights=True
)

train_model_lvl_2: tf.keras.callbacks.Callback = EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE,
    restore_best_weights=True
)

model_checkpoint_lvl_1: tf.keras.callbacks.Callback = ModelCheckpoint(
    filepath=os.path.join(model_dir, f"cortex-{MODEL_TYPE.lower()}-lvl-1.keras"),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)
model_checkpoint_lvl_2: tf.keras.callbacks.Callback = ModelCheckpoint(
    filepath=os.path.join(model_dir, f"cortex-{MODEL_TYPE.lower()}-lvl-2.keras"),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)


In [43]:
def load_existing_model(filepath):
    if os.path.exists(filepath):
        try:
            return load_model(filepath)
        except Exception as e:
            print(f"Could not load model from {filepath}: {e}")
    return None

def evaluate_model(model, val_data):
    _, acc = model.evaluate(val_data, verbose=0)
    return acc


def save_model_metadata(model, history, model_name, output_dir,train_data, val_data):
    history_path = os.path.join(output_dir, f"{model_name}_history.json")
    with open(history_path, 'w') as f:
        json.dump(history.history, f)

    summary_path = os.path.join(output_dir, f"{model_name}_summary.txt")
    with open(summary_path, 'w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))

    arch_path = os.path.join(output_dir, f"{model_name}_architecture.png")
    try:
        plot_model(model, to_file=arch_path, show_shapes=True, show_layer_names=True)
    except Exception as e:
        print("Could not generate model diagram:", e)
        
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    # Plot Accuracy
    if "accuracy" in history.history:
        axs[0].plot(history.history["accuracy"], label="train acc")
        axs[0].plot(history.history["val_accuracy"], label="val acc")
        axs[0].set_ylim(0, 1)
        axs[0].set_title("Training Accuracy")
        axs[0].set_xlabel("Epoch")
        axs[0].set_ylabel("Accuracy")
        axs[0].legend()

    # Plot Loss
    if "loss" in history.history:
        axs[1].plot(history.history["loss"], label="train loss")
        axs[1].plot(history.history["val_loss"], label="val loss")
        axs[1].set_ylim(0, 1)
        axs[1].set_title("Training Loss")
        axs[1].set_xlabel("Epoch")
        axs[1].set_ylabel("Loss")
        axs[1].legend()

    # Save the combined plot
    graph_path = os.path.join(output_dir, f"{model_name}_training_plot.png")
    plt.tight_layout()
    plt.savefig(graph_path)
    plt.close()

    # Save the Labels
    labels_path = os.path.join(output_dir, f"{model_name}_labels.json")
    with open(labels_path, 'w') as f:
        json.dump(train_data.class_indices, f)
    # Save the Configuration

    config = ""
    config += f"Model Name: {model_name}\n"
    config += f"Model Type: {MODEL_TYPE}\n"
    config += f"Epochs: {EPOCHS}\n"
    config += f"Batch Size: {BATCH_SIZE}\n"
    config += f"Learning Rate: {LEARNING_RATE}\n"
    config += f"Beta1: {BETA1}\n"
    config += f"Beta2: {BETA2}\n"
    config += f"Random State: {RANDOM_STATE}\n"
    config += f"Image Size: {IMG_SIZE}\n"
    config += f"Training Data Size: {len(train_data)*BATCH_SIZE}\n"
    config += f"Validation Data Size: {len(val_data)*BATCH_SIZE}\n"
    config_path = os.path.join(output_dir, f"{model_name}_config.txt")
    with open(config_path, 'w') as f:
        f.write(config)

In [44]:
existing_path_lvl_1: str = os.path.join(model_dir, f"cortex-{MODEL_TYPE.lower()}-lvl-1.keras")
existing_path_lvl_2: str = os.path.join(model_dir, f"cortex-{MODEL_TYPE.lower()}-lvl-2.keras")
old_model_lvl_1 = load_existing_model(existing_path_lvl_1)
old_model_lvl_2 = load_existing_model(existing_path_lvl_2)

if old_model_lvl_1 is not None:
    print("Evaluating existing Level 1 model...")
    evaluate_model(old_model_lvl_1, test_lvl_1_gen.gen)

if old_model_lvl_2 is not None:
    print("Evaluating existing Level 2 model...")
    evaluate_model(old_model_lvl_2, test_lvl_2_gen.gen)

In [45]:

history_lvl_1 = model_lvl_1.fit(
    train_lvl_1_gen.gen,
    validation_data=test_lvl_1_gen.gen,
    epochs=EPOCHS,
    callbacks=[train_model_lvl_1],
    verbose=1)

# Compare and conditionally save
new_accuracy = evaluate_model(model_lvl_1, test_lvl_1_gen.gen)
old_accuracy = evaluate_model(old_model_lvl_1, test_lvl_1_gen.gen) if old_model_lvl_1 else 0
print(f"Old Level 1 model accuracy: {old_accuracy:.4f}")
print(f"New Level 1 model accuracy: {new_accuracy:.4f}")
print(f"Improvement: {new_accuracy - old_accuracy:.4f}")

if new_accuracy > old_accuracy:
    print("✅ New Level 1 model is better. Saving the new model.")
    model_lvl_1.save(existing_path_lvl_1)
    save_model_metadata(
        model_lvl_1, history_lvl_1, "cortex-cnn-lvl-1", model_dir,train_data=train_lvl_1_gen.gen, val_data=test_lvl_1_gen.gen)
else:
    print("⚠️ Old Level 1 model is retained.")


Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.8220 - loss: 0.6619 - val_accuracy: 0.9115 - val_loss: 0.3189
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.9578 - loss: 0.2109 - val_accuracy: 0.9314 - val_loss: 0.2370
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.9724 - loss: 0.1573 - val_accuracy: 0.9497 - val_loss: 0.2045
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.9696 - loss: 0.1654 - val_accuracy: 0.9443 - val_loss: 0.1921
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.9797 - loss: 0.1218 - val_accuracy: 0.9680 - val_loss: 0.1533
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.9847 - loss: 0.1078 - val_accuracy: 0.9626 - val_loss: 0.1616
Old Level 1 model accuracy: 0.0000
New Level 1 model accuracy: 0.9680
Improvement: 0.9680
✅ New Level 1 model is better. Saving the new model.


In [46]:

history_lvl_2 = model_lvl_2.fit(
    train_lvl_2_gen.gen,
    validation_data=test_lvl_2_gen.gen,
    epochs=EPOCHS,
    callbacks=[train_model_lvl_2],
    verbose=1)

# Compare and conditionally save
new_accuracy = evaluate_model(model_lvl_2, test_lvl_2_gen.gen)
old_accuracyz = evaluate_model(old_model_lvl_2, test_lvl_2_gen.gen) if old_model_lvl_2 else 0

print(f"Old Level 2 model accuracy: {old_accuracy:.4f}")
print(f"New Level 2 model accuracy: {new_accuracy:.4f}")
print(f"Improvement: {new_accuracy - old_accuracy:.4f}")

if new_accuracy > old_accuracy:
    print("✅ New Level 2 model is better. Saving the new model.")
    model_lvl_2.save(existing_path_lvl_2)
    save_model_metadata(
        model_lvl_2, history_lvl_2, "cortex-cnn-lvl-2", model_dir,
        train_data=train_lvl_2_gen.gen, val_data=test_lvl_2_gen.gen)
else:
    print("⚠️ Old Level 2 model is retained.")


Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.4405 - loss: 1.3646 - val_accuracy: 0.7201 - val_loss: 0.7879
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.7576 - loss: 0.6829 - val_accuracy: 0.7452 - val_loss: 0.7320
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.8311 - loss: 0.5386 - val_accuracy: 0.7475 - val_loss: 0.7281
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.8603 - loss: 0.4890 - val_accuracy: 0.8322 - val_loss: 0.4779
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.9047 - loss: 0.3647 - val_accuracy: 0.8482 - val_loss: 0.5046
Old Level 2 model accuracy: 0.0000
New Level 2 model accuracy: 0.8322
Improvement: 0.8322
✅ New Level 2 model is better. Saving the new model.


# Analysis